In [63]:
import edgar

df = edgar.get_company_facts('wmt')
for y in list(filter(lambda x: 'Q' not in x, df.columns)):
    for q in range(1, 5):
        qtr_str = f"{y}Q{q}"
        if qtr_str in df.columns and q < 4:
            df[y] -= df[qtr_str]
        elif q == 4:
            df[qtr_str] = df[y]
            df.drop(y, axis=1, inplace=True)
df.loc['Gross Profit'] = df.loc['Revenue']-df.loc['COGS']
scale = {k: 1000000 for k in df.index}
scale['EPS'] = 1
df = df.div(scale, axis=0)
df

,CY2008Q2,CY2008Q3,CY2009Q1,CY2009Q2,CY2009Q3,CY2009Q4,CY2010Q1,CY2010Q2,CY2010Q3,CY2010Q4,...,CY2022Q2,CY2022Q3,CY2023Q1,CY2023Q2,CY2023Q3,CY2007Q4,CY2008Q4,CY2020Q4,CY2021Q4,CY2022Q4
Revenue,101546.00,97619.0,93471.00,100168.00,98667.00,112826.00,99097.00,103016.00,101239.00,118497.00,...,152859.00,152813.00,152301.00,161632.00,160804.00,373821.00,201922.00,152079.00,152871.00,164048.00
COGS,77118.00,73621.0,70395.00,75056.00,73915.00,84740.00,74618.00,77438.00,75819.00,87071.00,...,115838.00,115613.00,115284.00,121850.00,121183.00,284137.00,153202.00,115261.00,115522.00,125423.00
SG&A,19411.00,19432.0,18637.00,19891.00,20016.00,21433.00,19456.00,20098.00,20522.00,21285.00,...,30167.00,34505.00,30777.00,32466.00,33419.00,70934.00,38703.00,31331.00,31462.00,33064.00
Taxes,1826.00,1690.0,1603.00,1870.00,1702.00,1981.00,1822.00,1958.00,1505.00,2294.00,...,1497.00,336.00,792.00,2674.00,272.00,6889.00,-10649.00,1415.00,1149.00,3093.00
Other Income Loss,130.00,113.0,117.00,107.00,114.00,175.00,143.00,151.00,154.00,156.00,...,-2.00,31.00,223.00,162.00,190.00,406.00,256.00,83.00,71.00,-466.00
Shares,3958.00,3944.0,3930.00,3900.00,3861.00,-7814.00,3781.00,3707.00,3631.00,-7449.00,...,2745.00,2711.00,2704.00,2703.00,2703.00,4072.00,-3951.00,-5699.00,-5633.00,-5487.00
EPS,0.87,0.8,0.76,0.89,0.81,1.25,0.87,0.97,0.95,1.68,...,1.88,-0.66,0.62,2.92,0.17,3.13,1.72,-0.72,1.27,2.31
Operating Income,9392.00,8543.0,8330.00,9508.00,8700.00,12347.00,9181.00,9937.00,9201.00,14216.00,...,12001.00,928.00,8136.00,15369.00,6845.00,35089.00,18712.00,3479.00,9520.00,11370.00
Pretax Income,5357.00,4836.0,4750.00,5456.00,4967.00,6945.00,5266.00,5705.00,5095.00,7472.00,...,6644.00,-1431.00,2688.00,10727.00,915.00,20158.00,10674.00,12856.00,-4985.00,8902.00
Income,3449.00,3138.0,2996.00,3472.00,3144.00,4758.00,3301.00,3596.00,3436.00,6056.00,...,5149.00,-1798.00,1673.00,7891.00,453.00,12731.00,6794.00,-2091.00,3562.00,6275.00
